# Data Validation — Healthcare Insurance Claims
This notebook validates schema, missingness, business rules, and sanity stats for `data/raw/claims.csv`.


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path


In [5]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd()
if not (ROOT / "data/raw/claims.csv").exists():
    ROOT = ROOT.parent

DATA_PATH = ROOT / "data/raw/claims.csv"

print("Notebook CWD:", Path.cwd())
print("Resolved data path:", DATA_PATH)

assert DATA_PATH.exists(), f"Missing file: {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
df.head()


Notebook CWD: /workspaces/Healthcare-Insurance-claims-project/notebooks
Resolved data path: /workspaces/Healthcare-Insurance-claims-project/data/raw/claims.csv


,claim_id,patient_id,insurance_provider,service_date,submission_date,decision_date,payment_date,claim_amount,approved_amount,claim_status,denial_reason,days_to_decision,days_to_payment
0,CLM-100000,PT-93810,Medicare,2024-05-20,2024-05-22,2024-06-11,2024-06-29,205.20,177.10,Approved,NaN,20.0,18.0
1,CLM-100001,PT-14165,Medicare,2024-04-21,2024-04-23,2024-05-02,2024-05-24,84.51,80.91,Approved,NaN,9.0,22.0
2,CLM-100002,PT-64987,Medicare,2024-10-28,2024-10-31,2024-11-08,NaN,81.10,0.00,Denied,Duplicate Claim,8.0,NaN
3,CLM-100003,PT-54597,Medicare,2024-04-20,2024-04-23,2024-05-07,2024-05-31,144.59,141.28,Approved,NaN,14.0,24.0
4,CLM-100004,PT-44671,Medicare Advantage - Blue Cross,2024-08-23,2024-08-28,2024-09-02,2024-09-27,236.79,220.88,Approved,NaN,5.0,25.0


## 1) Basic shape & columns

In [6]:
print('Shape:', df.shape)
print('Columns:', list(df.columns))
df.info()

Shape: (1200, 13)
Columns: ['claim_id', 'patient_id', 'insurance_provider', 'service_date', 'submission_date', 'decision_date', 'payment_date', 'claim_amount', 'approved_amount', 'claim_status', 'denial_reason', 'days_to_decision', 'days_to_payment']
<class 'pandas.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   claim_id            1200 non-null   str    
 1   patient_id          1200 non-null   str    
 2   insurance_provider  1200 non-null   str    
 3   service_date        1200 non-null   str    
 4   submission_date     1200 non-null   str    
 5   decision_date       1030 non-null   str    
 6   payment_date        799 non-null    str    
 7   claim_amount        1200 non-null   float64
 8   approved_amount     1200 non-null   float64
 9   claim_status        1200 non-null   str    
 10  denial_reason       231 non-null    str    
 11  days_to_dec

## 2) Parse dates & numeric columns

In [7]:
date_cols = ['service_date','submission_date','decision_date','payment_date']
for c in date_cols:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')

num_cols = ['claim_amount','approved_amount','days_to_decision','days_to_payment']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

df[date_cols + num_cols].head()

,service_date,submission_date,decision_date,payment_date,claim_amount,approved_amount,days_to_decision,days_to_payment
0,2024-05-20,2024-05-22,2024-06-11,2024-06-29,205.20,177.10,20.0,18.0
1,2024-04-21,2024-04-23,2024-05-02,2024-05-24,84.51,80.91,9.0,22.0
2,2024-10-28,2024-10-31,2024-11-08,NaT,81.10,0.00,8.0,NaN
3,2024-04-20,2024-04-23,2024-05-07,2024-05-31,144.59,141.28,14.0,24.0
4,2024-08-23,2024-08-28,2024-09-02,2024-09-27,236.79,220.88,5.0,25.0


## 3) Missingness report

In [8]:
missing = (df.isna().mean()*100).sort_values(ascending=False)
missing.to_frame('missing_%').head(20)

,missing_%
denial_reason,80.750000
payment_date,33.416667
days_to_payment,33.416667
decision_date,14.166667
days_to_decision,14.166667
submission_date,0.000000
service_date,0.000000
insurance_provider,0.000000
patient_id,0.000000
claim_id,0.000000


## 4) Business rule checks (RCM logic)
- Pending: decision/payment can be null
- Approved: must have decision_date & payment_date, approved_amount > 0
- Denied: must have decision_date, payment_date should be null, approved_amount == 0


In [9]:
def count_bad(mask, label):
    n = int(mask.sum())
    print(f'{label}: {n}')
    return n

status = df['claim_status'].astype(str)

pending = status.eq('Pending')
approved = status.eq('Approved')
denied = status.eq('Denied')

# Approved must have decision & payment
bad_approved_missing_dates = approved & (df['decision_date'].isna() | df['payment_date'].isna())
count_bad(bad_approved_missing_dates, 'Approved claims missing decision/payment dates')

# Approved must have approved_amount > 0
bad_approved_amt = approved & ~(df['approved_amount'] > 0)
count_bad(bad_approved_amt, 'Approved claims with approved_amount <= 0')

# Denied should have decision_date and no payment_date
bad_denied_dates = denied & (df['decision_date'].isna() | df['payment_date'].notna())
count_bad(bad_denied_dates, 'Denied claims with missing decision_date OR has payment_date')

# Denied/Pending should have approved_amount == 0
bad_nonapproved_amt = (denied | pending) & ~(df['approved_amount'].fillna(0).eq(0))
count_bad(bad_nonapproved_amt, 'Denied/Pending claims with approved_amount != 0')

# approved_amount <= claim_amount
bad_amt_relation = df['approved_amount'].fillna(0) > df['claim_amount'].fillna(0)
count_bad(bad_amt_relation, 'Rows where approved_amount > claim_amount')

Approved claims missing decision/payment dates: 0
Approved claims with approved_amount <= 0: 0
Denied claims with missing decision_date OR has payment_date: 0
Denied/Pending claims with approved_amount != 0: 0
Rows where approved_amount > claim_amount: 0


0

## 5) Date ordering checks
Expected: service_date ≤ submission_date ≤ decision_date ≤ payment_date (where applicable)

In [10]:
bad_service_sub = df['service_date'].notna() & df['submission_date'].notna() & (df['service_date'] > df['submission_date'])
bad_sub_dec = df['submission_date'].notna() & df['decision_date'].notna() & (df['submission_date'] > df['decision_date'])
bad_dec_pay = df['decision_date'].notna() & df['payment_date'].notna() & (df['decision_date'] > df['payment_date'])

print('service_date > submission_date:', int(bad_service_sub.sum()))
print('submission_date > decision_date:', int(bad_sub_dec.sum()))
print('decision_date > payment_date:', int(bad_dec_pay.sum()))

service_date > submission_date: 0
submission_date > decision_date: 0
decision_date > payment_date: 0


## 6) Validate derived columns (days_to_decision / days_to_payment)
Checks whether the numeric columns match the date differences (tolerates nulls).

In [12]:
# Recompute expected day differences
calc_dtd = (df["decision_date"] - df["submission_date"]).dt.days
calc_dtp = (df["payment_date"] - df["decision_date"]).dt.days

# Use nullable integer dtype so NAs are allowed
dtd = df["days_to_decision"].astype("Int64")
dtp = df["days_to_payment"].astype("Int64")
calc_dtd_i = calc_dtd.astype("Int64")
calc_dtp_i = calc_dtp.astype("Int64")

# Only compare where BOTH sides are present
mismatch_dtd = dtd.notna() & calc_dtd_i.notna() & (dtd != calc_dtd_i)
mismatch_dtp = dtp.notna() & calc_dtp_i.notna() & (dtp != calc_dtp_i)

print("days_to_decision mismatches:", int(mismatch_dtd.sum()))
print("days_to_payment mismatches:", int(mismatch_dtp.sum()))

# Show a few mismatched examples (if any)
if mismatch_dtd.any():
    display(df.loc[mismatch_dtd, ["claim_id","submission_date","decision_date","days_to_decision"]].head(10))
if mismatch_dtp.any():
    display(df.loc[mismatch_dtp, ["claim_id","decision_date","payment_date","days_to_payment"]].head(10))


days_to_decision mismatches: 0
days_to_payment mismatches: 0


## 7) Quick sanity stats (dashboard-ready)

In [13]:
print(df['claim_status'].value_counts(dropna=False))
print('\nTop providers:')
print(df['insurance_provider'].value_counts().head(10))

approved_df = df[df['claim_status']=='Approved']
print('\nAvg days_to_payment (Approved):', round(approved_df['days_to_payment'].mean(), 2))
print('Avg days_to_decision (Approved):', round(approved_df['days_to_decision'].mean(), 2))

denied_df = df[df['claim_status']=='Denied']
print('\nTop denial reasons:')
print(denied_df['denial_reason'].value_counts().head(10))

claim_status
Approved    799
Denied      231
Pending     170
Name: count, dtype: int64

Top providers:
insurance_provider
Medicare                           533
Medicare Advantage - Aetna         256
Medicare Advantage - United        175
Private - Other                    124
Medicare Advantage - Blue Cross    112
Name: count, dtype: int64

Avg days_to_payment (Approved): 15.25
Avg days_to_decision (Approved): 11.49

Top denial reasons:
denial_reason
Eligibility Issue        55
Service Not Covered      53
Incorrect Coding         46
Duplicate Claim          44
Missing Documentation    33
Name: count, dtype: int64
